# Введение в обработку естественного языка

## Урок 16. Курсовой проект. Создание чат-бота в Telegram

Реализовать сценарии поведения для диалоговой системы:

* сценарий болталка

* покупка товара

* погода в городе

In [1]:
# !pip install pymorphy2
# !pip install pandarallel
# !pip install gdown
# !pip install num2words

# # Скачать файл Otvety.txt по ссылке  (1,7G)
# # https://drive.google.com/file/d/1DQL9ybca4USImUDaxxHmkIZNmClKBtKG
# !gdown --id 1DQL9ybca4USImUDaxxHmkIZNmClKBtKG

In [2]:
import pandas as pd
import numpy as np
import string
import pickle
import annoy
import mmap
import re
import os

from pymorphy2 import MorphAnalyzer
from gensim.models import Word2Vec, FastText
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from num2words import num2words
    
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [4]:
with open('Otvety.txt', 'r', encoding='utf-8') as f:
    for i in range(10):
        print(f.readline())



---

вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 

хомячка.... 

мужика, йопаря, собачку и 50 кошек))). 

Общение !). 

паучка. 

Да пол мне бы памыть!<br>А таг то ни чо. Типа ни каво!. 

я тут вообще что бы пообщаться..... 

А мне советовали сиси завести...))). 



In [5]:
# Преобразование файла аопросов-ответов в строчный вид
if not os.path.isfile('prepared_answers.txt'):

    question = None
    written = False

    with open("prepared_answers.txt", "w", encoding='utf-8') as fout:  
        with open("Otvety.txt", "r", encoding='utf-8') as fin:
            for line in tqdm(fin, ncols=80):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue

In [6]:
with open('prepared_answers.txt', 'r', encoding='utf-8') as f:
    for i in range(10):
        print(f.readline())

	вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 

Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .	меня вобще прикалывает эта тема :). 

Что делать, сегодня нашёл 2 миллиона рублей? .	Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. Можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. Выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!. 

Эбу в двенашке называется Итэлма что за эбу? .	ЭБУ — электронный блок управления двигателем автомобиля, его другое название — контроллер. Он принимает информацию от 

In [7]:
# Функция для обработки текста

def preprocess_txt(line):
    
    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    spls = re.sub(r'(<[^<]+?>)|(www.\S+)|(http\S+)|([0-9]+[-]?[\S]+)|([^а-яА-ЯёЁ0-9\s])', '', line)
    
    words = []
    for word in spls.split():
        if word.isdigit():
            words.append(num2words(word, lang='ru'))
        else:
            words.append(word)
    # spls = ' '.join(words)
    spls = " ".join(i.strip() for i in words if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in stop_words and i != ""]
    
    return spls

In [8]:
# Обработка текста

sentences = []
morpher = MorphAnalyzer()
exclude = set(string.punctuation)
stop_words = set(stopwords.words('russian'))

file_path_from = 'prepared_answers.txt'
file_path_to = 'Otvety2.txt'

if not os.path.isfile(file_path_to):
    
    N = get_num_lines(file_path_from)
    with open(file_path_to, 'w', encoding='utf-8') as fileto:
        with open(file_path_from, 'r', encoding='utf-8') as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences.append(spls)
                fileto.write(' '.join(spls)+'\n')

In [9]:
# Загрузить результат

sentences = []

file_path_from = 'Otvety2.txt'
if os.path.isfile(file_path_from):  
    N = get_num_lines(file_path_from) 
    with open(file_path_to, mode = 'r', encoding='utf-8') as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences.append(line.split())

  0%|          | 0/1163342 [00:00<?, ?it/s]

In [10]:
vec = []
_ = [vec.extend(x)  for x in sentences[:1000]]
vec = list(set(vec))
vec.sort()
vec[:10]

['а5',
 'аа',
 'аааааааа',
 'абдулл',
 'абзац',
 'абракадабра',
 'абрамович',
 'абсолютно',
 'ава',
 'авангард']

In [11]:
#df['text'] = value_counts(normalize=True)

In [12]:
sentences[:3]

[['вопрос', 'тдв', 'давно', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой',
  'вобщий',
  'прикалывать',
  'тема'],
 ['делать',
  'сегодня',
  'найти',
  'миллион',
  'рубль',
  'это',
  'счастие',
  'действительно',
  'свалиться',
  'хороший',
  'пойти',
  'милиция',
  'заявить',
  'находка',
  'деньга',
  'просто',
  'тероть',
  'самый',
  'интересный',
  'неприменный',
  'искать',
  'поверьте',
  'найти',
  'видеть',
  'подобный',
  'жизнь',
  'нарваться',
  'бабушка',
  'который',
  'хотеть',
  'помочий',
  'внук',
  'покупка',
  'квартира',
  'бандит',
  'который',
  'разговаривать',
  'иначе',
  'бабушка',
  'милиция',
  'выбор',
  'ещё',
  'шанс',
  'это',
  'подарок',
  'выше',
  'котрый',
  'никто',
  'спросить',
  'хороший',
  'отдать',
  'хотяб',
  'благотворительность',
  'дабы',
  'спугнуть',
  'удача']]

In [13]:
# Обучим модель FastText

file_path_from = 'ft_model'
if not os.path.isfile(file_path_from):  
    
    sentences = [i for i in tqdm(sentences) if len(i) > 2]
    modelFT = FastText(sentences=sentences, min_count=1, window=5)
    modelFT.save("ft_model")

In [14]:
# Загрузить модель

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [15]:
#Размерность вектора модели. Если установить 100 — каждое слово в корпусе будет представлено в виде 100-мерного вектора, и т.п.
#Наименьшее допустимое количество символов в слове, для которого будет создаваться векторное представление; так можно убрать частотные, но не очень значимые слова типа союзов и предлогов.
#Размер окна. Этот параметр задает, сколько соседних слов считается частью контекста. Если выставить 40, то алгоритм возьмет 40 слов спереди от слова и 40 слов сзади от слова;

In [16]:
list(set(stop_words))[:20]

['более',
 'мой',
 'во',
 'может',
 'потому',
 'и',
 'совсем',
 'был',
 'когда',
 'моя',
 'ничего',
 'что',
 'тогда',
 'к',
 'конечно',
 'она',
 'еще',
 'тебя',
 'до',
 'мне']

In [17]:
stop_words = set(stopwords.words('russian'))

In [18]:
# Создаем Индексы для вопросов-ответов

file_path_from = 'speaker.ann'
if not os.path.isfile(file_path_from):  
    morpher = MorphAnalyzer()
    sw = set(stop_words)
    exclude = set(string.punctuation)
    modelFT = FastText.load("ft_model")
    ft_index = annoy.AnnoyIndex(100 ,'angular')

    index_map = {}
    counter = 0
    with open("prepared_answers.txt", "r", encoding='utf-8') as f:
        for line in tqdm(f):
            n_ft = 0
            spls = line.split("\t")
            index_map[counter] = spls[1]
            question = preprocess_txt(spls[0])
            vector_ft = np.zeros(100)
            for word in question:
                if word in modelFT.wv:
                    vector_ft += modelFT.wv[word]
                    n_ft += 1
            if n_ft > 0:
                vector_ft = vector_ft / n_ft
            ft_index.add_item(counter, vector_ft)

            counter += 1

            if counter > 50_000:
                break

    ft_index.build(10)
    ft_index.save('speaker.ann')
    
    with open("index_speaker.pkl", "wb") as f:
        pickle.dump(index_map, f)

In [19]:
#  Загрузим индексы
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')
index_map = pd.read_pickle("index_speaker.pkl")

In [20]:
np.random.permutation(100)

array([86, 23, 77, 61, 10,  1, 90, 91, 58, 44, 21, 51, 98, 45, 85, 29, 82,
       26, 43, 89, 25, 55, 59,  2, 11, 71, 12, 70, 41, 27, 39, 88, 48, 95,
       38, 52, 17, 81, 66, 97, 30, 64,  3, 14, 49,  6, 96, 80, 37, 57, 79,
       72, 74,  4, 94, 36, 78, 67, 54, 68,  5, 13, 46, 75, 47, 99, 60,  8,
       34, 42, 18, 22, 92, 56, 16, 73, 32, 28, 76, 24, 20, 87, 84, 62, 65,
       35,  7, 19, 31, 50, 53, 83, 15,  9, 63,  0, 33, 69, 40, 93])

In [21]:
#  Пример получения индексов
a = ft_index.get_nns_by_vector(np.random.permutation(100), 5, include_distances=True)
a

([46761, 14837, 39796, 47038, 22494],
 [1.1474858522415161,
  1.1532166004180908,
  1.1532303094863892,
  1.1533552408218384,
  1.1540521383285522])

In [22]:
[index_map[x] for x in a[0]]

['Обычно так и бывает. В любом случае в мачо, который об этом треплется, даже если он действительно мачо, ничего хорошего нет. \n',
 'С двуличными людьми.. \n',
 'Лайнер приземлился, <br> Город "Здравствуй! "<br> мне сказал. <br> Я вернулась раньше, <br> И об этом ты не<br> знал. <br> Я к тебе спешила, <br> Я примчалась на<br> такси. <br> Ты открыл мне<br> дверь, <br> И вдруг войти не<br> пригласил.. .<br> <br> И глаза свои ты<br> опустил, <br> И даже бледным<br> стал. <br> И за твоим плечом<br> вижу я свою<br> подругу.. .<br> И смущенно так<br> сказал. <br> Что ты меня никак<br> не ждал. <br> Ну как ты мог? <br> Ведь мы же так<br> верили друг<br> другу.. .<br> <br> Я прошла на кухню, <br> Руки мелко так<br> дрожат. <br> Ты стоял в дверях, <br> Ее пальто в руках<br> держал. <br> К вам я повернулась<br> и сказала тихо ей: <br> "Ты здесь<br> оставайся, <br> Видно ты ему<br> нужней. " <br> <br> Я сбежала вниз, <br> Ты мне вдогонку<br> что-то закричал. <br> А я ведь знала, ты<br> изменил м

In [23]:
# https://gbcdn.mrgcdn.ru/uploads/asset/5209459/attachment/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv

In [24]:
# Создадим модель продуктовых данных

shop_data = pd.read_csv("ProductsDataset.csv")
#shop_data = shop_data.iloc[:5000, :]

shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].progress_apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, новый, носить, реал, красивый,..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, брюки, новый, знать, мерило, п..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, размерсостояние, отличное..."


In [25]:
# Подготовка для создания модели для определения домена данных

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [26]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
# Вопрос-ответный домен
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in tqdm(idxs)]
# Продуктовый домен
positive_texts = [" ".join(val) for val in tqdm(shop_data['text'].values)]

  0%|          | 0/25480 [00:00<?, ?it/s]

  0%|          | 0/35548 [00:00<?, ?it/s]

In [27]:
# ВО = 0, Прод = 1

dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [29]:
# Модель

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression(max_iter=200).fit(x_train_vec, y_train)

In [30]:
# Качество

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9505161396034737

In [31]:
# Добавим IDF взвешивание (для каждого слова найдем IDF вес)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [33]:
np.mean(tfidf_vect.idf_)

10.64556485451944

In [34]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [35]:
len(idfs.keys())

91133

In [36]:
list(idfs.keys())[:5]

['детский', 'плащ', 'продавать', 'хороший', 'состояние']

In [37]:
list(idfs.values())[:5]

[10.69734450361617,
 10.69734450361617,
 10.18651887985018,
 10.004197323056227,
 9.39806151948591]

In [38]:
# Создаем Индексы для продуктовых данных

file_path_from = 'shop.ann'
if not os.path.isfile(file_path_from):  
    
    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
        vector_ft = np.zeros(100)
        for word in shop_data.loc[i, "text"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_shop.add_item(counter, vector_ft)
        counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save('shop.ann')

    file_path_from = 'index_shop.pkl'
    if not os.path.isfile(file_path_from):  
    
        with open("index_shop.pkl", "wb") as f:
            pickle.dump(index_map_shop, f)


In [39]:
# Загрузим индексы

midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann') 

index_map_shop = pd.read_pickle("index_shop.pkl")

In [40]:
# Основная функция преобразования текста в вектор х100

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [41]:
# Пример получения индекса

ft_index_shop.get_nns_by_vector(np.ones(100)*20, 5, include_distances=True)

([35248, 6445, 32285, 6631, 33471],
 [1.1567997932434082,
  1.1702076196670532,
  1.175757884979248,
  1.1810858249664307,
  1.1830313205718994])

# Создание своего бота в телеграмм

In [42]:
from dotenv import load_dotenv
import os

from aiogram import Bot
from aiogram import Dispatcher
from aiogram import executor
from aiogram import types
from aiogram.dispatcher import FSMContext
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.contrib.fsm_storage.memory import MemoryStorage

import nest_asyncio
import asyncio

from pyowm.owm import OWM
import requests

In [43]:
# Определяем, к какому классу относится запрос и отправляем результат

def textMessage(message):
    input_txt = preprocess_txt(message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    # Если True - то выполняем сценарий покупки
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
        return f"Для вас у нас есть:\n<b>title:</b> <em>{title}</em>\n{image}"
    else:
        # Считаем вектор для сценария общения
        vect_ft = embed_txt(input_txt, {}, 1)
        ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    
    # Если полученное сообщение "звучит расплывчато"
    if distances[0] > 0.9:
        return "Не понимаю"
    
    # Сценарий общения
    return index_map[ft_index_val[0]]

In [44]:
# Получить данные о погоде

def get_wearher(place):

    api_weather = os.environ.get('api_weather')  # token для api.openweathermap.org
    url = 'https://api.openweathermap.org/data/2.5/weather?q=' + place + '&units=metric&lang=ru&appid=' + api_weather
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        name = data['name']
        temp = data['main']['temp']
        desc = data['weather'][0]['description']
        res = f'В городе {name} сейчас {desc}\nТемпература: {temp} °C'
    else:
        res = 'Мне не известен этот город'
    return res

In [45]:
# Настроим бота для подключения к telegram

load_dotenv()

storage = MemoryStorage()
bot = Bot(token=os.environ.get('token'))
dp = Dispatcher(bot, storage=storage)

# Машина состояний
class Form(StatesGroup):
    flag = State()

    
# Обработка сообщений пользователя

@dp.message_handler(state=Form.flag)
async def reply_weather(message: types.Message, state: FSMContext):
    await message.answer(get_wearher(message.text))
    await state.finish()

@dp.message_handler()
async def send_welcome(message: types.Message, state: FSMContext):
    if 'погода' in ' '.join(preprocess_txt(message.text)):
        await Form.flag.set()
        await message.reply('Могу подсказать погоду в одном из городов, если введёте название')
    else:
        await message.answer(textMessage(message), parse_mode='html')
        
nest_asyncio.apply()

In [46]:
# # Запуск бота

# async def main() -> None:
#     # And the run events dispatching
#     await dp.start_polling(bot)

# asyncio.run(main())

Бот доступен для тестирования по адресу в Telegram **@sdrtyherthrth_bot**